In [22]:
import muspy as mp
import json
import glob
from harmony_tokenizers import ChordSymbolTokenizer

In [ ]:
#examining nested dirs
letter_dirs = glob.glob('./hooktheory_data/xml/*')
theory_band_genres = {}

for l_dir in letter_dirs:
    #get band dir
    band_dir = glob.glob(l_dir+'/*')
    #for every band
    for b_dir in band_dir:
        #create for each band a dict with genres
        band_dict = {}
        #get track dir
        track_dir = glob.glob(b_dir+'/*')
        cnt = 0
        band_genres = []
        for t_dir in track_dir:
            #open json
            with open (t_dir+'/song_info.json') as f:
                md = json.load(f)
            #get genres
            band_genres.extend(md['genres'])

        # Remove duplicates from band_genres
        band_genres = list(set(band_genres))

        #store them for the band for later reference
        theory_band_genres[b_dir] = band_genres

print('Number of bands:', len(theory_band_genres))

# Print the  genres for the first 10 bands
for i, (key, value) in enumerate(theory_band_genres.items()):
    if i >= 10: 
        break
    print(f"{key}: {value}")

Number of bands: 5115
./hooktheory_data/xml\a\a-day-to-remember: ['Metal', 'Rock']
./hooktheory_data/xml\a\a-g-cook: ['Singer-Songwriter', 'Pop']
./hooktheory_data/xml\a\a-great-big-world: ['Singer-Songwriter', 'Pop']
./hooktheory_data/xml\a\a-ha: ['Pop']
./hooktheory_data/xml\a\a-love-like-pi: ['Alternative', 'Indie']
./hooktheory_data/xml\a\a-lovely-war: []
./hooktheory_data/xml\a\a-perfect-circle: ['Alternative', 'Rock']
./hooktheory_data/xml\a\a-r-rahman: ['Soundtrack', 'House']
./hooktheory_data/xml\a\a-sunny-day-in-glasgow: []
./hooktheory_data/xml\a\a-teens: ['Pop']


In [21]:
all_pieces = []
cnt_errs = 0

#parse them

for b_dir in theory_band_genres:
    #replace xml to mid
    b_dir = b_dir.replace('xml', 'midis' )
    #get track dir
    track_dir = glob.glob(b_dir+'/*/*.mid')
    for t_dir in track_dir:
        try:
            g = mp.read_midi(t_dir)
            all_pieces.append(g)
        except Exception as e:
            # Catch and display the error
            print(f"Error parsing file: {t_dir}")
            print(f"Error details: {e}")
            cnt_errs += 1

print('Total number of excepetions:', cnt_errs)

Total number of excepetions: 0


In [25]:
piece = all_pieces[0]
print(piece)

Music(metadata=Metadata(schema_version='0.2', title='None', source_filename='intro-and-verse.mid', source_format='midi'), resolution=220, tempos=[Tempo(time=0, qpm=129.00007740004645)], key_signatures=[KeySignature(time=0, root=0, mode='major')], time_signatures=[TimeSignature(time=0, numerator=4, denominator=4)], lyrics=[Lyric(time=0, lyric='C'), Lyric(time=440, lyric='C'), Lyric(time=660, lyric='C'), ...], tracks=[Track(program=0, is_drum=False, notes=[Note(time=3300, pitch=41, duration=55, velocity=100), Note(time=3355, pitch=43, duration=55, velocity=100), Note(time=3410, pitch=47, duration=110, velocity=100), ...]), Track(program=0, is_drum=False, notes=[Note(time=0, pitch=48, duration=440, velocity=100), Note(time=0, pitch=52, duration=440, velocity=100), Note(time=0, pitch=55, duration=440, velocity=100), ...])])


In [24]:
# prepare stats
stats = {}

print('ChordSymbolTokenizer')
chordSymbolTokenizer = ChordSymbolTokenizer()
print('len(chordSymbolTokenizer.vocab): ', len(chordSymbolTokenizer.vocab))
stats['ChordSymbolTokenizer'] = {
    'vocab_size': len(chordSymbolTokenizer.vocab),
    'seq_lens': []
}

toks_cs = chordSymbolTokenizer(all_pieces)
print('example sentence length: ', len(toks_cs['tokens'][0]))

ChordSymbolTokenizer
len(chordSymbolTokenizer.vocab):  370


AttributeError: 'Track' object has no attribute 'harmony'